In [23]:
import yfinance as yf

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import clone

In [24]:
df_apple_to_csv = yf.download("AAPL", start="2020-01-01", end="2025-08-13")
df_microsoft_to_csv = yf.download("MSFT", start="2020-01-01", end="2025-08-13")
df_amazon_to_csv = yf.download("AMZN", start="2020-01-01", end="2025-08-13")
df_nvidia_to_csv = yf.download("NVDA", start="2020-01-01", end="2025-08-13")


apple_path = 'datasets/apple.csv'
microsoft_path = 'datasets/microsoft.csv'
amazon_path = 'datasets/amazon.csv'
nvidia_path = 'datasets/nvidia.csv'

df_apple_to_csv.to_csv(apple_path)
df_microsoft_to_csv.to_csv(microsoft_path)
df_amazon_to_csv.to_csv(amazon_path)
df_nvidia_to_csv.to_csv(nvidia_path)

df_apple = pd.read_csv(apple_path)
df_microsoft = pd.read_csv(microsoft_path)
df_amazon = pd.read_csv(amazon_path)
df_nvidia = pd.read_csv(nvidia_path)

C:\Users\acer\AppData\Local\Temp\ipykernel_8680\126190450.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_apple_to_csv = yf.download("AAPL", start="2020-01-01", end="2025-08-13")
[*********************100%***********************]  1 of 1 completed


C:\Users\acer\AppData\Local\Temp\ipykernel_8680\126190450.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_microsoft_to_csv = yf.download("MSFT", start="2020-01-01", end="2025-08-13")
[*********************100%***********************]  1 of 1 completed
C:\Users\acer\AppData\Local\Temp\ipykernel_8680\126190450.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_amazon_to_csv = yf.download("AMZN", start="2020-01-01", end="2025-08-13")
[*********************100%***********************]  1 of 1 completed
C:\Users\acer\AppData\Local\Temp\ipykernel_8680\126190450.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_nvidia_to_csv = yf.download("NVDA", start="2020-01-01", end="2025-08-13")
[*********************100%***********************]  1 of 1 completed


In [25]:
df_apple.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,72.53852081298828,72.59889913772518,71.29231130597061,71.5458973217052,135480400
3,2020-01-03,71.83329772949219,72.59406313642577,71.60869213351766,71.76567430155144,146322800
4,2020-01-06,72.40569305419922,72.44433607127044,70.70302703365871,70.95420295942078,118387200


In [26]:
df_microsoft.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,MSFT,MSFT,MSFT,MSFT,MSFT
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,153.04226684570312,153.14707784853206,150.8603108363314,151.28907783375524,22622100
3,2020-01-03,151.1365966796875,152.40385148053937,150.6030187717304,150.85076175678734,21116200
4,2020-01-06,151.52732849121094,151.59403303057528,149.1262122356397,149.66932792275708,20813700


In [27]:
df_amazon.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,AMZN,AMZN,AMZN,AMZN,AMZN
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,94.90049743652344,94.90049743652344,93.2074966430664,93.75,80580000
3,2020-01-03,93.74849700927734,94.30999755859375,93.2249984741211,93.2249984741211,75288000
4,2020-01-06,95.14399719238281,95.18450164794922,93.0,93.0,81236000


In [28]:
df_apple['Date'] = df_apple['Price']
df_apple = df_apple.drop(index=[0,1], columns='Price')

df_microsoft['Date'] = df_microsoft['Price']
df_microsoft = df_microsoft.drop(index=[0,1], columns='Price')

df_amazon['Date'] = df_amazon['Price']
df_amazon = df_amazon.drop(index=[0,1], columns='Price')

df_nvidia['Date'] = df_nvidia['Price']
df_nvidia = df_nvidia.drop(index=[0,1], columns='Price')

In [29]:
df_apple.head()

,Close,High,Low,Open,Volume,Date
2,72.53852081298828,72.59889913772518,71.29231130597061,71.5458973217052,135480400,2020-01-02
3,71.83329772949219,72.59406313642577,71.60869213351766,71.76567430155144,146322800,2020-01-03
4,72.40569305419922,72.44433607127044,70.70302703365871,70.95420295942078,118387200,2020-01-06
5,72.06513977050781,72.67133299565539,71.84536183598526,72.41532954893665,108872000,2020-01-07
6,73.22442626953125,73.52631792855652,71.76810120120939,71.76810120120939,132079200,2020-01-08


In [30]:
df_microsoft.head()

,Close,High,Low,Open,Volume,Date
2,153.04226684570312,153.14707784853206,150.8603108363314,151.28907783375524,22622100,2020-01-02
3,151.1365966796875,152.40385148053937,150.6030187717304,150.85076175678734,21116200,2020-01-03
4,151.52732849121094,151.59403303057528,149.1262122356397,149.66932792275708,20813700,2020-01-06
5,150.145751953125,152.13714723225632,149.89802336904734,151.8036681235651,21634100,2020-01-07
6,152.53729248046875,153.21380259511494,150.4982536880769,151.43201531179338,27746500,2020-01-08


In [31]:
df_amazon.head()

,Close,High,Low,Open,Volume,Date
2,94.90049743652344,94.90049743652344,93.2074966430664,93.75,80580000,2020-01-02
3,93.74849700927734,94.30999755859375,93.2249984741211,93.2249984741211,75288000,2020-01-03
4,95.14399719238281,95.18450164794922,93.0,93.0,81236000,2020-01-06
5,95.34300231933594,95.69450378417969,94.60199737548828,95.2249984741211,80898000,2020-01-07
6,94.59850311279297,95.55000305175781,94.3219985961914,94.9020004272461,70160000,2020-01-08


In [32]:
df_nvidia.head()

,Close,High,Low,Open,Volume,Date
2,5.971746444702148,5.971746444702148,5.892342616375051,5.942872368471369,237536000,2020-01-02
3,5.876163482666016,5.9199728720181835,5.827126877093122,5.852018586378897,205384000,2020-01-03
4,5.900805473327637,5.906032689136935,5.756683394717106,5.782819473763594,262636000,2020-01-06
5,5.972244739532471,6.018045311441172,5.884128406356931,5.9291821653038275,314856000,2020-01-07
6,5.9834465980529785,6.024766771198286,5.927938401487843,5.968013727979196,277108000,2020-01-08


In [33]:
df_apple.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1410 entries, 2 to 1411
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Close   1410 non-null   object
 1   High    1410 non-null   object
 2   Low     1410 non-null   object
 3   Open    1410 non-null   object
 4   Volume  1410 non-null   object
 5   Date    1410 non-null   object
dtypes: object(6)
memory usage: 66.2+ KB


In [34]:
df_apple.describe()

,Close,High,Low,Open,Volume,Date
count,1410,1410,1410,1410,1410,1410
unique,1394,1410,1410,1410,1408,1410
top,139.26513671875,230.8000030517578,227.07000732421875,228.00999450683594,90956700,2025-08-12
freq,2,1,1,1,2,1


In [35]:
df_apple.isna().sum()

Close     0
High      0
Low       0
Open      0
Volume    0
Date      0
dtype: int64

In [36]:
#INITIAL
dfs = [df_apple, df_amazon, df_microsoft, df_nvidia]
window = 14
short_period = 12
long_period = 26
signal_line_period = 9
years = list(range(2020,2026))
stocks = ['AAPL', 'AMZN', 'MSFT', 'NVDA']
X_train, X_test, y_train, y_test = None, None, None, None

In [37]:
#CONVERTING TO CORRECT FEATURE TYPES
def convert_type(df):
    df['Close'] = df['Close'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Open'] = df['Open'].astype(float)
    df['Volume'] = df['Volume'].astype(float)
    df['Date'] = pd.to_datetime(df['Date'])

In [38]:
#FEATURE ENGINEERING
def engineer_features(df):
    df['Price Change'] = df['Close'].diff().fillna(0)

    df['Gain'] = df.loc[df['Price Change'] > 0, 'Price Change']
    df['Loss'] = -df.loc[df['Price Change'] < 0, 'Price Change']

    df['Gain'] = df['Gain'].fillna(0)
    df['Loss'] = df['Loss'].fillna(0)

    df['Average Gain'] = df['Gain'].rolling(window=window).mean().fillna(0)
    df['Average Loss'] = df['Loss'].rolling(window=window).mean().fillna(0)

    df['RS'] = df['Average Gain'] / df['Average Loss']

    df['RSI'] = 100 - (100 / (1 + df['RS']))

    df['Short EMA'] = df['Close'].ewm(span=short_period,adjust=False).mean()
    df['Long EMA'] = df['Close'].ewm(span=long_period, adjust=False).mean()

    df['MACD'] = df['Short EMA'] - df['Long EMA']

    df['Signal Line'] = df['MACD'].ewm(span=signal_line_period, adjust=False).mean()
    
    df['MACD Histogram'] = df['MACD'] - df['Signal Line']
    
    df['Year'] = df['Date'].dt.year

In [39]:
#DROPPING NULL AND UNIMPORTANT COLUMNS/ROWS
def drop_features_and_na(df):
    df = df.drop(columns=['Gain', 'Loss'])
    df = df.dropna()
    return df

In [40]:
#PRINTING SEABORN PLOTS
def print_plots(df, stock):
    for year in years:
        sns.relplot(data=df.loc[df['Year'] == year], x='Date', y='RSI', kind='line')
        plt.title(f'RSI trend for year {year} Stock {stock}')
        plt.figure(figsize=(20,10))
        plt.show()
        
        sns.relplot(data=df.loc[df['Year'] == year], x='Date', y='MACD', kind='line')
        plt.title(f'MACD trend for year {year} Stock {stock}')
        plt.figure(figsize=(20,10))
        plt.show()

In [41]:
#MODEL TRAINING AND EVALUATION
def split_data(df):
    X = df.drop(columns=['Date', 'Close'])
    y = df['Close']

    return train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
def create_pipeline():
    model = Pipeline([
        ('scaler', StandardScaler()),
        ('lr', LinearRegression())
        ])
    return clone(model)

In [43]:
def evaluate(model):
    kfold = KFold(n_splits=6, random_state=30, shuffle=True)

    params = {'lr__n_jobs': [10,20,50]}
    cv = clone(GridSearchCV(estimator=model,param_grid=params,cv=kfold))

    cv.fit(X_train, y_train)
    
    return {"Best parameter": cv.best_params_, "Best score": cv.best_score_}, cv

In [44]:
#MAIN OUTPUT
models = []

for i,df in enumerate(dfs):
    convert_type(df)
    engineer_features(df)
    df = drop_features_and_na(df)
    
    X_train, X_test, y_train, y_test = split_data(df)
    
    model = create_pipeline()
    
    evaluation, cv = evaluate(model)
    
    models.append(cv)
    
    print(f"{stocks[i]}:")
    print(df.head())
    print()
    
    # print_plots(df, stocks[i]) TODO: must implement merged stocks for good plotting visuals

AAPL:
        Close       High        Low       Open       Volume       Date  \
15  76.728760  77.281820  76.634566  76.941285  101832400.0 2020-01-22   
16  77.098274  77.177970  76.233653  76.781892  104472000.0 2020-01-23   
17  76.876099  78.088493  76.685301  77.344635  146537600.0 2020-01-24   
18  74.615547  75.296609  73.632586  74.883623  161940000.0 2020-01-27   
19  76.726364  76.897837  75.398041  75.497062  162234000.0 2020-01-28   

    Price Change  Average Gain  Average Loss        RS        RSI  Short EMA  \
15      0.272903      0.508216      0.208913  2.432667  70.868134  75.349471   
16      0.369514      0.534610      0.208913  2.559006  71.902270  75.618518   
17     -0.222176      0.534610      0.174410  3.065254  75.401289  75.811992   
18     -2.260551      0.493724      0.335878  1.469953  59.513403  75.627923   
19      2.110817      0.644497      0.311552  2.068664  67.412524  75.796914   

     Long EMA      MACD  Signal Line  MACD Histogram  Year  
15  74.

In [45]:
for i, model in enumerate(models):
    y_pred = model.predict(X_test)

    print(f"Predicted values for {stocks[i]}:")
    print(y_pred[0:10])

    print(f"Correct values for {stocks[i]}:")
    print(y_test[0:10])

Predicted values for AAPL:
[ 19.54622547 115.39786002 141.70603341  45.52842506  27.79084988
  12.81338215  11.79786879 178.48211284 133.58622517  48.33641145]
Correct values for AAPL:
378      19.960102
1280    116.641174
1234    141.917297
994      46.606258
493      28.109703
687      13.245834
708      12.180056
1407    179.419998
1354    134.370667
1003     48.808285
Name: Close, dtype: float64
Predicted values for AMZN:
[ 19.61357437 115.91246053 142.01567539  45.78796204  27.94867487
  12.98517625  12.00407863 178.76894513 133.77138872  48.62982506]
Correct values for AMZN:
378      19.960102
1280    116.641174
1234    141.917297
994      46.606258
493      28.109703
687      13.245834
708      12.180056
1407    179.419998
1354    134.370667
1003     48.808285
Name: Close, dtype: float64
Predicted values for MSFT:
[ 18.57606958 114.24674413 140.89717994  43.51764427  25.36895906
   9.51710453   9.32232857 178.22805118 133.3062938   47.70529706]
Correct values for MSFT:
378      

In [46]:
#TODO: BELOW THIS CELL SHOULD FIX MODEL SELECTION, PIPELINE, PREDICTING FOR EACH STOCKS

In [47]:
# model = Pipeline([
#     ('scaler', StandardScaler()),
#     ('lr', LinearRegression())
# ])

In [48]:
# kfold = KFold(n_splits=6, random_state=30, shuffle=True)

# params = {'lr__n_jobs': [10,20,50]}
# cv = GridSearchCV(estimator=model,param_grid=params,cv=kfold)

# cv.fit(X_train, y_train)

# print("Best parameter:", cv.best_params_)
# print("Best score:", cv.best_score_)


In [49]:
# y_pred = cv.predict(X_test)

# print("Predicted values respectively:",y_pred[0:10])

# print("Correct values:")
# print(y_test[0:10])